# NLP12- Transformer: 멋진 챗봇 만들기
# 목차

> LMS 따라해보기 

1. 번역 데이터 준비
2. 번역 모델 만들기
3. 번역 성능 측정하기 
  * (1) BLEU Score
  * (2) Beam Search Decoder
4. 데이터 부풀리기




> 프로젝트: 멋진 챗봇 만들기

* STEP 1. 데이터 다운로드
* STEP 2. 데이터 정제
* STEP 3. 데이터 토큰화
* STEP 4. Augmentation
* STEP 5. 데이터 벡터화
* STEP 6. 훈련하기
* STEP 7. 성능 측정하기

## 회고

# LMS 따라해보기

In [1]:
! pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sentencepiece as spm
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

import re
import os
import random
import math

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

print(tf.__version__)

2.9.2


##1. 번역 데이터 준비



### (1) 데이터 준비

sentencepiece 설치
 

In [3]:
zip_path = tf.keras.utils.get_file(
    'spa-eng.zip',
    origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True
)

중복된 데이터가 있을 수 있으니 list와 set을 사용해 처리

In [4]:
file_path = os.path.dirname(zip_path)+"/spa-eng/spa.txt"

with open(file_path, "r") as f:
    spa_eng_sentences = f.read().splitlines()

spa_eng_sentences = list(set(spa_eng_sentences)) 
total_sentence_count = len(spa_eng_sentences)
print("Example:", total_sentence_count)

for sen in spa_eng_sentences[0:100:20]: 
    print(">>", sen)

Example: 118964
>> The coat doesn't have any pockets.	El abrigo no tiene bolsillos.
>> No one agreed with me.	Nadie estuvo de acuerdo conmigo.
>> Please phone him.	Llámalo, por favor.
>> It's awfully cold this evening.	Esta tarde hace muchísimo frío.
>> What exactly am I paying for?	¿Qué estoy pagando exactamente?


전처리 함수

In [5]:

def preprocess_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'[" "]+', " ", sentence)   # 공백 여러개 -> 공백 하나: 변경 
    sentence = sentence.strip()
    return sentence

모든 데이터에 대해서 같은 전처리

In [6]:
spa_eng_sentences = list(map(preprocess_sentence, spa_eng_sentences))

train_test_split


In [7]:
test_sentence_count = total_sentence_count // 200
print("Test Size: ", test_sentence_count)
print("\n")

train_spa_eng_sentences = spa_eng_sentences[:-test_sentence_count]
test_spa_eng_sentences = spa_eng_sentences[-test_sentence_count:]

print("Train Example:", len(train_spa_eng_sentences))
for sen in train_spa_eng_sentences[0:100:20]: 
    print(">>", sen)
print("\n")
print("Test Example:", len(test_spa_eng_sentences))
for sen in test_spa_eng_sentences[0:100:20]: 
    print(">>", sen)

Test Size:  594


Train Example: 118370
>> the coat doesn't have any pockets.	el abrigo no tiene bolsillos.
>> no one agreed with me.	nadie estuvo de acuerdo conmigo.
>> please phone him.	llámalo, por favor.
>> it's awfully cold this evening.	esta tarde hace muchísimo frío.
>> what exactly am i paying for?	¿qué estoy pagando exactamente?


Test Example: 594
>> tom made fun of mary.	tom se burló de mary.
>> why don't they talk to me?	¿por qué no hablan conmigo?
>> we saw a terrible movie last night.	anoche vimos una película horrenda.
>> i have fewer students in my class this year than last year.	este año tengo menos estudiantes en mi clase que el año pasado.
>> i usually toss my loose change into my desk drawer.	normalmente echo la calderilla en el cajón de mi escritorio.


영어와 스페인어를 분리

: 영어 문장과 스페인어 문장이 tab으로 연결되어 있으니 split('\t') 사용
* tab 이전: 영어 문장

* tab 이후: 스페인어 문장

In [8]:
def split_spa_eng_sentences(spa_eng_sentences):
    spa_sentences = []
    eng_sentences = []
    for spa_eng_sentence in tqdm(spa_eng_sentences):
        eng_sentence, spa_sentence = spa_eng_sentence.split('\t')
        spa_sentences.append(spa_sentence)
        eng_sentences.append(eng_sentence)
    return eng_sentences, spa_sentences

In [9]:
train_eng_sentences, train_spa_sentences = split_spa_eng_sentences(train_spa_eng_sentences)
print(len(train_eng_sentences))
print(train_eng_sentences[0])
print('\n')
print(len(train_spa_sentences))
print(train_spa_sentences[0])

  0%|          | 0/118370 [00:00<?, ?it/s]

118370
the coat doesn't have any pockets.


118370
el abrigo no tiene bolsillos.


In [10]:
test_eng_sentences, test_spa_sentences = split_spa_eng_sentences(test_spa_eng_sentences)
print(len(test_eng_sentences))
print(test_eng_sentences[0])
print('\n')
print(len(test_spa_sentences))
print(test_spa_sentences[0])

  0%|          | 0/594 [00:00<?, ?it/s]

594
tom made fun of mary.


594
tom se burló de mary.


### (2) 토큰화

`generate_tokenizer()` :  Sentencepiece 기반의 토크나이저를 생성

* 두 언어 단어 사전을 공유
* 영어와 스페인어 모두 알파벳으로 이뤄지는 데다가 같은 인도유럽어족이기 때문에 기대할 수 있는 효과가 크다

In [11]:
def generate_tokenizer(corpus,
                       vocab_size,
                       lang="spa-eng",
                       pad_id=0,   # pad token의 일련번호
                       bos_id=1,  # 문장의 시작을 의미하는 bos token(<s>)의 일련번호
                       eos_id=2,  # 문장의 끝을 의미하는 eos token(</s>)의 일련번호
                       unk_id=3):   # unk token의 일련번호
    file = "./%s_corpus.txt" % lang
    model = "%s_spm" % lang

    with open(file, 'w') as f:
        for row in corpus: f.write(str(row) + '\n')

    import sentencepiece as spm
    spm.SentencePieceTrainer.Train(
        '--input=./%s --model_prefix=%s --vocab_size=%d'\
        % (file, model, vocab_size) + \
        '--pad_id==%d --bos_id=%d --eos_id=%d --unk_id=%d'\
        % (pad_id, bos_id, eos_id, unk_id)
    )

    tokenizer = spm.SentencePieceProcessor()
    tokenizer.Load('%s.model' % model)

    return tokenizer


In [12]:
VOCAB_SIZE = 20000
tokenizer = generate_tokenizer(train_eng_sentences + train_spa_sentences, VOCAB_SIZE, 'spa-eng')
tokenizer.set_encode_extra_options("bos:eos")  # 문장 양 끝에 <s> , </s> 추가

True

In [13]:
def make_corpus(sentences, tokenizer):
    corpus = []
    for sentence in tqdm(sentences):
        tokens = tokenizer.encode_as_ids(sentence)
        corpus.append(tokens)
    return corpus


In [14]:
eng_corpus = make_corpus(train_eng_sentences, tokenizer)
spa_corpus = make_corpus(train_spa_sentences, tokenizer)

  0%|          | 0/118370 [00:00<?, ?it/s]

  0%|          | 0/118370 [00:00<?, ?it/s]

In [15]:
print(train_eng_sentences[0])
print(eng_corpus[0])
print('\n')
print(train_spa_sentences[0])
print(spa_corpus[0])

the coat doesn't have any pockets.
[1, 9, 1774, 140, 7, 17, 39, 238, 6196, 0, 2]


el abrigo no tiene bolsillos.
[1, 19, 1761, 14, 101, 6193, 0, 2]


`pad_sequences()` : list 자료형도 단숨에 패딩 작업을 해주는 멋진 함수 

In [16]:
MAX_LEN = 50
enc_ndarray = tf.keras.preprocessing.sequence.pad_sequences(eng_corpus, maxlen=MAX_LEN, padding='post')
dec_ndarray = tf.keras.preprocessing.sequence.pad_sequences(spa_corpus, maxlen=MAX_LEN, padding='post')


배치 크기의 텐서 생성 :모델 훈련에 사용될 수 있도록 영어와 스페인어 데이터를 묶음

In [17]:
BATCH_SIZE = 64
train_dataset = tf.data.Dataset.from_tensor_slices((enc_ndarray, dec_ndarray)).batch(batch_size=BATCH_SIZE)


##2. 번역 모델 생성 : Transformer



### (1) 모델 설계 

>Positional Encoding


In [18]:
# Positional Encoding 구현
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, (2*(i//2)) / np.float32(d_model))

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table


> Mask

In [19]:
# Mask  생성하기
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_lookahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_enc_mask = generate_padding_mask(src)

    dec_lookahead_mask = generate_lookahead_mask(tgt.shape[1])
    dec_tgt_padding_mask = generate_padding_mask(tgt)
    dec_mask = tf.maximum(dec_tgt_padding_mask, dec_lookahead_mask)

    return enc_mask, dec_enc_mask, dec_mask

> Multi-head Attention

In [20]:
# Multi Head Attention 구현
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
        

    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x

    
    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
                        
        out = self.combine_heads(out)
        out = self.linear(out)
            
        return out, attention_weights

> Position-wise Feed Forward Network

In [21]:
# Position-wise Feed Forward Network 구현
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
            
        return out

> Encoder Layer

In [22]:
# Encoder의 레이어 구현
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        '''
        Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn

>Decoder Layer

In [23]:
# Decoder 레이어 구현
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        '''
        Masked Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        '''
        Multi-Head Attention
        '''
        residual = out
        out = self.norm_2(out)
        # Q, K, V 순서에 주의하세요!
        out, dec_enc_attn = self.enc_dec_attn(Q=out, K=enc_out, V=enc_out, mask=dec_enc_mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

>Encoder

In [24]:
# Encoder 구현
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
    
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        
        return out, enc_attns

>Decoder

In [25]:
# Decoder 구현
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, dec_enc_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

> Transformer 전체 모델 조립

In [26]:

class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out

        
    def call(self, enc_in, dec_in, enc_mask, dec_enc_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        
        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, dec_enc_mask, dec_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns

### (2) 모델 훈련

> 모델 인스턴스 생성

In [27]:
# 주어진 하이퍼파라미터로 Transformer 인스턴스 생성

transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)
		
d_model = 512


>Learning Rate Scheduler 정의


In [28]:
# Learning Rate Scheduler 구현
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

>Learning Rate & Optimizer

In [29]:

# Learning Rate 인스턴스 선언 & Optimizer 구현
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)


>Loss Function 정의

In [30]:
# Loss Function 정의
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


>Train Step 정의


In [31]:
# Train Step 정의
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns


>훈련시키기

In [32]:
# 훈련시키기
EPOCHS = 3

for epoch in range(EPOCHS):
    total_loss = 0
    
    dataset_count = tf.data.experimental.cardinality(train_dataset).numpy()
    tqdm_bar = tqdm(total=dataset_count)
    for step, (enc_batch, dec_batch) in enumerate(train_dataset):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_batch,
                    dec_batch,
                    transformer,
                    optimizer)

        total_loss += batch_loss
        
        tqdm_bar.set_description_str('Epoch %2d' % (epoch + 1))
        tqdm_bar.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (step + 1)))
        tqdm_bar.update()

  0%|          | 0/1850 [00:00<?, ?it/s]

  0%|          | 0/1850 [00:00<?, ?it/s]

  0%|          | 0/1850 [00:00<?, ?it/s]

##3. Translation Evaluation
  * (1) BLEU Score
  * (2) Beam Search Decoder

### (1) BLEU Score

>NLTK를 활용한 BLEU Score

`NLTK(Natural Language Tool Kit)` : 자연어 처리에 큰 도움이 되는  라이브러리. 
* nltk 가 BLEU Score를 지원


In [33]:
# 아래 두 문장을 바꿔가며 테스트 해보세요
reference = "많 은 자연어 처리 연구자 들 이 트랜스포머 를 선호 한다".split()
candidate = "적 은 자연어 학 개발자 들 가 트랜스포머 을 선호 한다 요".split()

print("원문:", reference)
print("번역문:", candidate)
print("BLEU Score:", sentence_bleu([reference], candidate))

원문: ['많', '은', '자연어', '처리', '연구자', '들', '이', '트랜스포머', '를', '선호', '한다']
번역문: ['적', '은', '자연어', '학', '개발자', '들', '가', '트랜스포머', '을', '선호', '한다', '요']
BLEU Score: 8.190757052088229e-155


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [34]:
print("1-gram:", sentence_bleu([reference], candidate, weights=[1, 0, 0, 0]))
print("2-gram:", sentence_bleu([reference], candidate, weights=[0, 1, 0, 0]))
print("3-gram:", sentence_bleu([reference], candidate, weights=[0, 0, 1, 0]))
print("4-gram:", sentence_bleu([reference], candidate, weights=[0, 0, 0, 1]))

1-gram: 0.5
2-gram: 0.18181818181818182
3-gram: 2.2250738585072626e-308
4-gram: 2.2250738585072626e-308


>BLEU Score 보정하기

:`SmoothingFunction()` 활용

In [35]:
def calculate_bleu(reference, candidate, weights=[0.25, 0.25, 0.25, 0.25]):
    return sentence_bleu([reference],
                         candidate,
                         weights=weights,
                         smoothing_function=SmoothingFunction().method1)  # smoothing_function 적용

print("BLEU-1:", calculate_bleu(reference, candidate, weights=[1, 0, 0, 0]))
print("BLEU-2:", calculate_bleu(reference, candidate, weights=[0, 1, 0, 0]))
print("BLEU-3:", calculate_bleu(reference, candidate, weights=[0, 0, 1, 0]))
print("BLEU-4:", calculate_bleu(reference, candidate, weights=[0, 0, 0, 1]))

print("\nBLEU-Total:", calculate_bleu(reference, candidate))

BLEU-1: 0.5
BLEU-2: 0.18181818181818182
BLEU-3: 0.010000000000000004
BLEU-4: 0.011111111111111112

BLEU-Total: 0.05637560315259291


>트랜스포머 모델의 번역 성능 측정

`translate()` 함수를 정의

In [36]:
def translate(tokens, model, src_tokenizer, tgt_tokenizer):
    padded_tokens = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=MAX_LEN,
                                                           padding='post')
    ids = []
    output = tf.expand_dims([tgt_tokenizer.bos_id()], 0)   
    for i in range(MAX_LEN):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(padded_tokens, output)

        predictions, _, _, _ = model(padded_tokens, 
                                      output,
                                      enc_padding_mask,
                                      combined_mask,
                                      dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if tgt_tokenizer.eos_id() == predicted_id:
            result = tgt_tokenizer.decode_ids(ids)  
            return result

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = tgt_tokenizer.decode_ids(ids)  
    return result


 `eval_bleu_single()` : 한 문장만 평가
 
 * 번역한 문장의 BLEU Score를 평가 

In [37]:
def eval_bleu_single(model, src_sentence, tgt_sentence, src_tokenizer, tgt_tokenizer, verbose=True):
    src_tokens = src_tokenizer.encode_as_ids(src_sentence)
    tgt_tokens = tgt_tokenizer.encode_as_ids(tgt_sentence)

    if (len(src_tokens) > MAX_LEN): return None
    if (len(tgt_tokens) > MAX_LEN): return None

    reference = tgt_sentence.split()
    candidate = translate(src_tokens, model, src_tokenizer, tgt_tokenizer).split()

    score = sentence_bleu([reference], candidate,
                          smoothing_function=SmoothingFunction().method1)

    if verbose:
        print("Source Sentence: ", src_sentence)
        print("Model Prediction: ", candidate)
        print("Real: ", reference)
        print("Score: %lf\n" % score)
        
    return score
        


In [38]:
# 인덱스를 바꿔가며 테스트해 보세요
test_idx = 0

eval_bleu_single(transformer, 
                 test_eng_sentences[test_idx], 
                 test_spa_sentences[test_idx], 
                 tokenizer, 
                 tokenizer)

Source Sentence:  tom made fun of mary.
Model Prediction:  ['tom', 'se', 'divirtió', 'mucho', 'a', 'mary', 'de', 'mary?']
Real:  ['tom', 'se', 'burló', 'de', 'mary.']
Score: 0.065006



0.06500593260343691

` eval_bleu() `: 전체 테스트 데이터에 대해서 평가

* eval_bleu_single을 이용해서 eval_bleu 함수

In [39]:
def eval_bleu(model, src_sentences, tgt_sentence, src_tokenizer, tgt_tokenizer, verbose=True):
    total_score = 0.0
    sample_size = len(src_sentences)
    
    for idx in tqdm(range(sample_size)):
        score = eval_bleu_single(model, src_sentences[idx], tgt_sentence[idx], src_tokenizer, tgt_tokenizer, verbose)
        if not score: continue
        
        total_score += score
    
    print("Num of Sample:", sample_size)
    print("Total Score:", total_score / sample_size)

In [40]:
eval_bleu(transformer, test_eng_sentences, test_spa_sentences, tokenizer, tokenizer, verbose=False)

  0%|          | 0/594 [00:00<?, ?it/s]

Num of Sample: 594
Total Score: 0.09369089364696591


### (2) Beam Search Decoder

`beam_search_decoder()` :

In [41]:
def beam_search_decoder(prob, beam_size):
    sequences = [[[], 1.0]]  # 생성된 문장과 점수를 저장

    for tok in prob:
        all_candidates = []

        for seq, score in sequences:
            for idx, p in enumerate(tok): # 각 단어의 확률을 총점에 누적 곱
                candidate = [seq + [idx], score * -math.log(-(p-1))]
                all_candidates.append(candidate)

        ordered = sorted(all_candidates,
                         key=lambda tup:tup[1],
                         reverse=True) # 총점 순 정렬
        sequences = ordered[:beam_size] # Beam Size에 해당하는 문장만 저장 

    return sequences

In [42]:
vocab = {
    0: "<pad>",
    1: "까요?",
    2: "커피",
    3: "마셔",
    4: "가져",
    5: "될",
    6: "를",
    7: "한",
    8: "잔",
    9: "도",
}

prob_seq = [[0.01, 0.01, 0.60, 0.32, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.75, 0.01, 0.01, 0.17],
            [0.01, 0.01, 0.01, 0.35, 0.48, 0.10, 0.01, 0.01, 0.01, 0.01],
            [0.24, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.68],
            [0.01, 0.01, 0.12, 0.01, 0.01, 0.80, 0.01, 0.01, 0.01, 0.01],
            [0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.11, 0.01, 0.01, 0.01],
            [0.70, 0.22, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]]

prob_seq = np.array(prob_seq)
beam_size = 3

result = beam_search_decoder(prob_seq, beam_size)

for seq, score in result:
    sentence = ""

    for word in seq:
        sentence += vocab[word] + " "

    print(sentence, "// Score: %.4f" % score)

커피 를 가져 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 42.5243
커피 를 마셔 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 28.0135
마셔 를 가져 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 17.8983


>Beam Search Decoder 작성 및 평가하기

* calc_prob() : 각 단어의 확률값을 계산
* beam_search_decoder() : Beam Search를 기반으로 동작
* beam_bleu() : 생성된 문장에 대해 BLEU Score를 출력

`calc_prob()` : 각 단어의 확률값을 계산

In [43]:
# calc_prob() 구현

def calc_prob(src_ids, tgt_ids, model):
    enc_padding_mask, combined_mask, dec_padding_mask = \
    generate_masks(src_ids, tgt_ids)

    predictions, enc_attns, dec_attns, dec_enc_attns =\
    model(src_ids, 
            tgt_ids,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask)
    
    return tf.math.softmax(predictions, axis=-1)


`beam_search_decoder()` : Beam Search를 기반으로 동작

In [44]:
# beam_search_decoder() 구현
def beam_search_decoder(sentence, 
                        src_len,
                        tgt_len,
                        model,
                        src_tokenizer,
                        tgt_tokenizer,
                        beam_size):
    tokens = src_tokenizer.encode_as_ids(sentence)
    
    src_in = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                            maxlen=src_len,
                                                            padding='post')

    pred_cache = np.zeros((beam_size * beam_size, tgt_len), dtype=np.int64)
    pred_tmp = np.zeros((beam_size, tgt_len), dtype=np.int64)

    eos_flag = np.zeros((beam_size, ), dtype=np.int64)
    scores = np.ones((beam_size, ))

    pred_tmp[:, 0] = tgt_tokenizer.bos_id()

    dec_in = tf.expand_dims(pred_tmp[0, :1], 0)
    prob = calc_prob(src_in, dec_in, model)[0, -1].numpy()

    for seq_pos in range(1, tgt_len):
        score_cache = np.ones((beam_size * beam_size, ))

        # init
        for branch_idx in range(beam_size):
            cache_pos = branch_idx*beam_size

            score_cache[cache_pos:cache_pos+beam_size] = scores[branch_idx]
            pred_cache[cache_pos:cache_pos+beam_size, :seq_pos] = \
            pred_tmp[branch_idx, :seq_pos]

        for branch_idx in range(beam_size):
            cache_pos = branch_idx*beam_size

            if seq_pos != 1:   # 모든 Branch를 로 시작하는 경우를 방지
                dec_in = pred_cache[branch_idx, :seq_pos]
                dec_in = tf.expand_dims(dec_in, 0)

                prob = calc_prob(src_in, dec_in, model)[0, -1].numpy()

            for beam_idx in range(beam_size):
                max_idx = np.argmax(prob)

                score_cache[cache_pos+beam_idx] *= prob[max_idx]
                pred_cache[cache_pos+beam_idx, seq_pos] = max_idx

                prob[max_idx] = -1

        for beam_idx in range(beam_size):
            if eos_flag[beam_idx] == -1: continue

            max_idx = np.argmax(score_cache)
            prediction = pred_cache[max_idx, :seq_pos+1]

            pred_tmp[beam_idx, :seq_pos+1] = prediction
            scores[beam_idx] = score_cache[max_idx]
            score_cache[max_idx] = -1

            if prediction[-1] == tgt_tokenizer.eos_id():
                eos_flag[beam_idx] = -1

    pred = []
    for long_pred in pred_tmp:
        zero_idx = long_pred.tolist().index(tgt_tokenizer.eos_id())
        short_pred = long_pred[:zero_idx+1]
        pred.append(short_pred)
    return pred

In [45]:
def calculate_bleu(reference, candidate, weights=[0.25, 0.25, 0.25, 0.25]):
    return sentence_bleu([reference],
                            candidate,
                            weights=weights,
                            smoothing_function=SmoothingFunction().method1)


`beam_bleu()` : 생성된 문장에 대해 BLEU Score를 출력

In [46]:
# beam_bleu() 구현
def beam_bleu(reference, ids, tokenizer):
    reference = reference.split()

    total_score = 0.0
    for _id in ids:
        candidate = tokenizer.decode_ids(_id.tolist()).split()
        score = calculate_bleu(reference, candidate)

        print("Reference:", reference)
        print("Candidate:", candidate)
        print("BLEU:", calculate_bleu(reference, candidate))

        total_score += score
        
    return total_score / len(ids)

In [47]:
# 인덱스를 바꿔가며 확인해 보세요
test_idx = 6

ids = \
beam_search_decoder(test_eng_sentences[test_idx],
                    MAX_LEN,
                    MAX_LEN,
                    transformer,
                    tokenizer,
                    tokenizer,
                    beam_size=5)

bleu = beam_bleu(test_spa_sentences[test_idx], ids, tokenizer)
print(bleu)

Reference: ['no', 'tengáis', 'tanta', 'prisa.']
Candidate: ['no', 'seáis', 'prisa,', '¿verdad?']
BLEU: 0.08034284189446518
Reference: ['no', 'tengáis', 'tanta', 'prisa.']
Candidate: ['no', 'se', 'pase', 'prisa,', '¿verdad?']
BLEU: 0.05372849659117709
Reference: ['no', 'tengáis', 'tanta', 'prisa.']
Candidate: ['no', 'teáis', 'prisa,', '¿verdad?']
BLEU: 0.08034284189446518
Reference: ['no', 'tengáis', 'tanta', 'prisa.']
Candidate: ['no', 'te', 'pase', 'prisa,', '¿verdad?']
BLEU: 0.05372849659117709
Reference: ['no', 'tengáis', 'tanta', 'prisa.']
Candidate: ['no', 'paseáis', 'prisa,', '¿verdad?']
BLEU: 0.08034284189446518
0.06969710377314994


##4. Data Augmenatation

In [48]:
import gensim.downloader as api

wv = api.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [49]:
wv.most_similar("banana")

[('bananas', 0.6691170930862427),
 ('mango', 0.580410361289978),
 ('pineapple', 0.5492371916770935),
 ('coconut', 0.5462779402732849),
 ('papaya', 0.541056752204895),
 ('fruit', 0.5218108296394348),
 ('growers', 0.4877638816833496),
 ('nut', 0.4839959144592285),
 ('peanut', 0.48062020540237427),
 ('potato', 0.4806118607521057)]

In [50]:
sample_sentence = "you know ? all you need is attention ."
sample_tokens = sample_sentence.split()

selected_tok = random.choice(sample_tokens)

result = ""
for tok in sample_tokens:
    if tok is selected_tok:
        result += wv.most_similar(tok)[0][0] + " "

    else:
        result += tok + " "

print("From:", sample_sentence)
print("To:", result)

From: you know ? all you need is attention .
To: you know ? all 'll need is attention . 


### Lexical Substitution 구현하기

대표적으로 사용되는 Embedding 모델은 word2vec-google-news-300 이지만 용량이 커서 다운로드에 많은 시간이 소요되므로 이번 실습엔 적합하지 않습니다. 우리는 적당한 사이즈의 모델인 glove-wiki-gigaword-300 을 사용할게요! 아래 소스를 실행해 사전 훈련된 Embedding 모델을 다운로드해 주세요.

In [51]:
# Lexical Substitution 구현하기
def lexical_sub(sentence, word2vec):
    res = ""
    toks = sentence.split()

    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[0][0]
        
    except:   # 단어장에 없는 단어
        return None

    for tok in toks:
        if tok is _from: res += _to + " "
        else: res += tok + " "

    return res


In [52]:
new_corpus = []

for old_src in tqdm(test_eng_sentences):
    new_src = lexical_sub(old_src, wv)
    if new_src is not None: 
        new_corpus.append(new_src)
    # Augmentation이 없더라도 원본 문장을 포함시킵니다
    new_corpus.append(old_src)

print(new_corpus[:10])

  0%|          | 0/594 [00:00<?, ?it/s]

['bob made fun of mary. ', 'tom made fun of mary.', "this is the better christmas i've had in years. ", "this is the best christmas i've had in years.", 'there being a great deal of conjecture as to what would happen. ', 'there was a great deal of conjecture as to what would happen.', 'i needs a stamp. ', 'i need a stamp.', 'of movie that tom and mary watched together had a happy ending. ', 'the movie that tom and mary watched together had a happy ending.']


# PROJECT : Chatbot

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
import gensim
import re
from tqdm.notebook import tqdm

print(np.__version__)
print(pd.__version__)
print(tf.__version__)
print(nltk.__version__)
print(gensim.__version__)

1.21.6
1.3.5
2.9.2
3.7
3.6.0


## STEP 1. 데이터 다운로드


  [songys/Chatbot_data](https://github.com/songys/Chatbot_data)
* Pandas로 ChatbotData.csv 파일 읽기
* 질문과 답변을 각각 questions, answers 변수에 나눠서 저장

In [2]:
chatbot_df = pd.read_csv('https://github.com/songys/Chatbot_data/raw/master/ChatbotData.csv')
chatbot_df

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [3]:
print('전체 샘플수 :', (len(chatbot_df)))

전체 샘플수 : 11823


In [4]:
chatbot_df.drop(("label"), axis="columns", inplace=True)
chatbot_df.head()

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.


In [5]:
# # 중복 시각화
chatbot_df[chatbot_df.duplicated()]

,Q,A
5527,결혼이나 하지 왜 자꾸 나한테 화 내냐구!,힘들겠네요.
5537,고백하고 후회하면 어떡하지,후회는 후회를 낳을뿐이에요. 용기 내세요.
5542,공부는 내 체질이 아닌 것 같아,확신이 없나봐요.
5704,기숙사 괜찮을까,혼자 사는 것보다 불편하겠죠.
5774,나는 좋은데 ….,현실의 벽에 부딪혔나봐요.
...,...,...
8764,환승 가능?,환승은 30분 안에
8780,회사 사람들이 아직도 불편해,회사에는 동료가 있을 뿐이에요.
8782,회사에는 왜 친구 같은 사람이 없을까,회사는 친구 사귀는 곳이 아니에요.
8789,후련하달까,후련하니 다행이에요.


In [6]:

# # 중복 샘플 확인
# print('Q 열에서 중복을 배제한 유일한 샘플의 수 :', chatbot_df['Q'].nunique())
# print('A 열에서 중복을 배제한 유일한 샘플의 수 :', chatbot_df['A'].nunique())

In [7]:
# ## inplace=True 를 설정하면 DataFrame 타입 값을 return 하지 않고 data 내부를 직접적으로 바꿉니다
# chatbot_df.drop_duplicates(subset = ['Q'], inplace=True)
# chatbot_df

In [8]:
question_f = chatbot_df['Q'].values.tolist()
print(question_f[:5])

['12시 땡!', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']


In [9]:
questions_list = []
for q in chatbot_df['Q']:
  questions_list.append(q)

print(questions_list[:5])

['12시 땡!', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']


In [10]:
answers_list = []
for a in chatbot_df['A']:
  answers_list.append(a)

print(answers_list[:5])

['하루가 또 가네요.', '위로해 드립니다.', '여행은 언제나 좋죠.', '여행은 언제나 좋죠.', '눈살이 찌푸려지죠.']


In [11]:
print(len(questions_list))
print(len(answers_list))

11823
11823


## STEP 2. 데이터 정제


`set() `
중복 제거 
* 데이터 쌍이 흐트러지지 않게 주의

In [12]:
# 중복 쌍으로 제거 
pair_unique = list(set(zip(questions_list,answers_list)))


print("중복 제거된 pair 데이터 수 : ", len(pair_unique))
print('\nexample : \n',pair_unique[:5])

중복 제거된 pair 데이터 수 :  11750

example : 
 [('거리가 먼데 고백하면 받아줄까?', '거리를 뛰어넘는 사랑이라면 가능하겟죠.'), ('후배보다 못해서 짜증나', '질투하지 마세요.'), ('널 사랑하지 않아', '직접 말해보세요.'), ('공부 좀 더 할 걸', '지금도 늦지 않았어요.'), ('조금 힘들어.', '쪼그만 아파하고 괜찮아지세요.')]


In [13]:
pair_unique[25]

('목이 뻑뻑해', '목감기 오려나봐요.')

In [14]:

# 다시 쌍 해제
questions, answers = zip(*pair_unique)

print("중복 제거된 questions 데이터수 : ", len(questions))
print("중복 제거된 answers 데이터수 : ", len(answers))

중복 제거된 questions 데이터수 :  11750
중복 제거된 answers 데이터수 :  11750


In [15]:
questions[25]

'목이 뻑뻑해'

In [16]:
answers[25]

'목감기 오려나봐요.'

`preprocess_sentence()`
> 
> 1. 영문자의 경우, 모두 소문자 변환
2. 영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 정규식을 활용하여 모두 제거


In [17]:

def preprocess_sentence(sentence):

    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)   #??
    sentence = re.sub(r'[" "]+', " ", sentence)         # 공백 여러개 공백하나로
    sentence = re.sub(r"[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9?.!,]+", " ", sentence) # 대소문자, 기본 특수기호 외는 공백하나로

    sentence = sentence.strip()     # 양쪽 공백 제거 

    # if s_token:
    #     sentence = '<start> ' + sentence

    # if e_token:
    #     sentence += ' <end>'

    return sentence


In [18]:
print(preprocess_sentence(questions[25]))
print(preprocess_sentence(answers[25]))

목이 뻑뻑해
목감기 오려나봐요 .


## STEP 3. 데이터 토큰화


토큰화:  KoNLPy의 mecab 클래스 사용

> `build_corpus()`
1. 소스 문장 데이터, 타겟 문장 데이터 입력
2. 데이터를 앞서 정의한 preprocess_sentence() 함수로 정제하고, 토큰화
3. 토큰화 : mecab.morphs()
4. 토큰의 개수 제한
5. 중복되는 문장 제외
* 소스 : 타겟 쌍을 비교하지 않고 소스는 소스대로 타겟은 타겟대로 검사합니다. 중복 쌍이 흐트러지지 않도록 유의하세요!


In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
!pip install konlpy

In [20]:
from konlpy.tag import Mecab

In [21]:
cleaned_que = []
cleaned_ans = []

# num_examples = 30000

for que, ans in zip(questions_list, answers_list):
    # eng, spa = pair.split("\t")

    cleaned_que.append(preprocess_sentence(que))
    cleaned_ans.append(preprocess_sentence(ans))


print(len(cleaned_que))
print(len(cleaned_ans))

11823
11823


In [22]:
cleaned_que[0]

'12시 땡 !'

In [23]:

# mecab = Mecab()

# que_corpus = []
# # ans_corpus = []

# for i in que_corpus: 
#     que_corpus.append(mecab.morphs(i))


# # ans_corpus = mecab.morphs(ans_corpus)

# print(len(que_corpus))

In [24]:
mecab = Mecab()

def tokenizing(src, tgt):
    
    src_corpus = []
    tgt_corpus = []
    
    mecab = Mecab()
    
    for i in range(len(cleaned_que)):

        src_tokens = mecab.morphs(cleaned_que[i])
        tgt_tokens = mecab.morphs(cleaned_ans[i])
        
        if src_tokens in src_corpus and tgt_tokens in tgt_corpus: continue
        
        src_corpus.append(src_tokens)
        tgt_corpus.append(tgt_tokens)
        
    return src_corpus, tgt_corpus

In [25]:
que_corpus, ans_corpus = tokenizing(cleaned_que, cleaned_ans)
print(len(que_corpus), len(ans_corpus))

11720 11720


In [26]:
print(que_corpus[25])
print(ans_corpus[25])

['가족', '여행', '어디', '로', '가', '지', '?']
['온', '가족', '이', '모두', '마음', '에', '드', '는', '곳', '으로', '가', '보', '세요', '.']


In [27]:
len(que_corpus)

11720

## STEP 4. Augmentation


 데이터 1만 개가량으로 적은 편: 
 
 > Lexical Substitution 적용
1. 한국어로 사전 훈련된 Embedding 모델(Korean (w)) 다운로드 후 ko.bin 파일 얻기
* [songys/Chatbot_data](https://github.com/songys/Chatbot_data)
2. lexical_sub() 함수 참고 : Kyubyong/wordvectors 모델로 데이터  Augmentation 진행

1)원본 que_corpus : 원본 ans_corpus

2)Augmentation que_corpus : 원본 ans_corpus  병렬 

3)원본 que_corpus : Augmentation ans_corpus  병렬

전체 데이터가 원래의 3배가량 증대  

In [28]:
import gensim
word2vec = gensim.models.Word2Vec.load(r'/content/drive/MyDrive/Aiffel_data/NLP/[NLP-12_2]/ko/ko.bin')

# model = gensim.models.KeyedVectors.load_word2vec_format(r'/content/drive/MyDrive/Aiffel_data/NLP/[NLP-12_2]/ko/ko.bin', binary=True)

In [29]:
a = word2vec.wv.most_similar("강아지")
a

[('고양이', 0.7290452718734741),
 ('거위', 0.7185635566711426),
 ('토끼', 0.7056223154067993),
 ('멧돼지', 0.6950401067733765),
 ('엄마', 0.6934334635734558),
 ('난쟁이', 0.6806551218032837),
 ('한마리', 0.6770296096801758),
 ('아가씨', 0.6750352382659912),
 ('아빠', 0.6729634404182434),
 ('목걸이', 0.6512460708618164)]

In [47]:
# Lexical Substitution 구현하기
import random
def lexical_sub(sentence, word2vec):
    res = ""
    #toks = sentence.split() mecab은 이미 split이 되어 tokenizing 처리 된다.
    toks = sentence

    try:
        _from = random.choice(toks)
        _to = word2vec.wv.most_similar(_from)[0][0]
        
    except:   # 단어장에 없는 단어
        # print(_from)
        return None

    for tok in toks:
        if tok is _from: res += _to + " "
        else: res += tok + " "

    return res


In [ ]:
new_que_corpus = []
new_ans_corpus = []

corpus_size= len(que_corpus)

# Augmentation된 que_corpus 와 원본 ans_corpus 가 병렬 -> 3배로 늘림
for i in tqdm(range(corpus_size)):
    que_augmented = lexical_sub(que_corpus[i], word2vec)
    ans = ans_corpus[i]
    
    if que_augmented is not None:
        new_que_corpus.append(que_augmented.split())
        new_ans_corpus.append(ans)
        
    else:
       
        continue
    
for i in tqdm(range(corpus_size)):
    que = que_corpus[i]
    ans_augmented = lexical_sub(ans_corpus[i], word2vec)
    
    if ans_augmented is not None:
        new_que_corpus.append(que)
        new_ans_corpus.append(ans_augmented.split())
       
    else:
       
        continue

In [49]:
print('증강된 질문 데이터셋 수 :', len(new_que_corpus))
print('증강된 대답 데이터셋 수 :', len(new_ans_corpus), '\n')

print(new_que_corpus[0])
print(new_ans_corpus[0], '\n')

print(new_que_corpus[50])
print(new_ans_corpus[50], '\n')

증강된 질문 데이터셋 수 : 20326
증강된 대답 데이터셋 수 : 20326 

['12', '시', '땡', '캐치']
['하루', '가', '또', '가', '네요', '.'] 

['함부로', '나', '를', '무시', '하', '는', '애', '가', '있', '어']
['콕', '집', '어서', '물', '어', '보', '세요', '.'] 



In [50]:
# 증강한 데이터셋 원래 데이터셋 하나로 합치기  
que_corpus = que_corpus + new_que_corpus
ans_corpus = ans_corpus + new_ans_corpus

In [51]:
print('기존 + 증강 질문 데이터셋 수 :', len(que_corpus))
print('기존 + 증강 대답 데이터셋 수 :', len(ans_corpus), '\n')

기존 + 증강 질문 데이터셋 수 : 32046
기존 + 증강 대답 데이터셋 수 : 32046 



## STEP 5. 데이터 벡터화

1. 타겟 데이터 전체에 \<start> 토큰과 \<end> 토큰 추가 :  ans_corpus 완성

* 챗봇 훈련 데이터의 가장 큰 특징 : 바로 소스 데이터와 타겟 데이터가 같은 언어를 사용 
* 이는 Embedding 층을 공유했을 때 많은 이점을 얻을 수 있습니다.

2. 
* 전체 데이터 단어 사전 구축: ans_corpus 와 que_corpus 결합
* 벡터화 :  enc_train 과 dec_train 

In [52]:
sample_data = ["12", "시", "땡", "!"]

print(["<start>"] + sample_data + ["<end>"])

['<start>', '12', '시', '땡', '!', '<end>']


In [53]:
# ans_corpus : 타겟 데이터 전체에 <start> 토큰과 <end> 토큰 추가 

tgt_corpus = []

for corpus in ans_corpus:
    tgt_corpus.append(["<start>"] + corpus + ["<end>"])

print(tgt_corpus[0])

ans_corpus = tgt_corpus
print(ans_corpus[0])

['<start>', '하루', '가', '또', '가', '네요', '.', '<end>']
['<start>', '하루', '가', '또', '가', '네요', '.', '<end>']


In [54]:
from collections import Counter

voc_data = que_corpus + ans_corpus

words = np.concatenate(voc_data).tolist()
counter = Counter(words)
counter = counter.most_common()
#counter = counter.most_common(30000-2) 
#len(counter.most_common()) :총개수 7942
vocab = ['<pad>', '<unk>'] + [key for key, _ in counter]
word_to_index = {word:index for index, word in enumerate(vocab)}
index_to_word = {index:word for word, index in word_to_index.items()}

❤️ 한줄코드 읽는법 공부해보기 

In [55]:
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index[word] if word in word_to_index else word_to_index['<unk>'] for word in sentence]

    # arr = []
    # for word in sentence:
    #   if word in word_to_index:
    #     arr.append(word_to_index[word])
    #   else:
    #     arr.append(word_to_index['unk'])

def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<unk>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

def vectorize(corpus, word_to_index):
    return [get_encoded_sentence(sen, word_to_index) for sen in corpus]

    data = []
    for sen in corpus:
        sen = get_encoded_sentence(sen, word_to_index)
        data.append(sen)
    return data
# data = []
# data= data.append(get_encoded_sentence(sen, word_to_index)) for sen in corpus



que_train = vectorize(que_corpus, word_to_index)
ans_train = vectorize(ans_corpus, word_to_index)

print(len(que_train))
print(len(ans_train),'\n')
print(que_train[0])
print(ans_train[0])

32046
32046 

[2552, 188, 4146, 95]
[3, 271, 9, 160, 9, 39, 2, 4]


In [56]:
from sklearn.model_selection import train_test_split

In [57]:
enc_tensor = tf.keras.preprocessing.sequence.pad_sequences(que_train, padding='post')
dec_tensor = tf.keras.preprocessing.sequence.pad_sequences(ans_train, padding='post')

enc_train, enc_val, dec_train, dec_val = \
train_test_split(enc_tensor, dec_tensor, test_size=0.01) # test set은 1%만

print(len(enc_train), len(enc_val), len(dec_train), len(dec_val))

31725 321 31725 321


In [58]:
BATCH_SIZE = 64
train_dataset = tf.data.Dataset.from_tensor_slices((enc_tensor, dec_tensor)).batch(batch_size=BATCH_SIZE)


## STEP 6. 훈련하기

* 데이터의 크기가 작으니 하이퍼파라미터를 튜닝해야 과적합 피함. 

가장 멋진 답변과 모델의 하이퍼파라미터를 제출

### 모델 설계

In [59]:
# Positional Encoding 구현
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, (2*(i//2)) / np.float32(d_model))

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table


In [60]:
# Mask  생성하기
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_lookahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_enc_mask = generate_padding_mask(src)

    dec_lookahead_mask = generate_lookahead_mask(tgt.shape[1])
    dec_tgt_padding_mask = generate_padding_mask(tgt)
    dec_mask = tf.maximum(dec_tgt_padding_mask, dec_lookahead_mask)

    return enc_mask, dec_enc_mask, dec_mask

In [61]:
# Multi Head Attention 구현
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
        

    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x

    
    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
                        
        out = self.combine_heads(out)
        out = self.linear(out)
            
        return out, attention_weights

In [62]:
# Position-wise Feed Forward Network 구현
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
            
        return out

In [63]:
# Encoder의 레이어 구현
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        '''
        Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn

In [64]:
# Decoder 레이어 구현
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        '''
        Masked Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        '''
        Multi-Head Attention
        '''
        residual = out
        out = self.norm_2(out)
        # Q, K, V 순서에 주의하세요!
        out, dec_enc_attn = self.enc_dec_attn(Q=out, K=enc_out, V=enc_out, mask=dec_enc_mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

In [65]:
# Encoder 구현
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
    
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        
        return out, enc_attns

In [66]:
# Decoder 구현
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, dec_enc_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

In [67]:
# Transformer 조립

class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out

        
    def call(self, enc_in, dec_in, enc_mask, dec_enc_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        
        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, dec_enc_mask, dec_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns

### 모델 훈련


In [111]:
# 주어진 하이퍼파라미터로 Transformer 인스턴스 생성
VOCAB_SIZE = len(counter)

transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)
		

In [101]:
# Learning Rate Scheduler 구현
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)


In [112]:
# Learning Rate 인스턴스 선언 & Optimizer 구현
d_model=512
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)


In [113]:
# Loss Function 정의
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask]
    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


In [114]:
# Train Step 정의
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns


In [105]:
# BATCH_SIZE = 64
# train_dataset = tf.data.Dataset.from_tensor_slices((enc_tensor, dec_tensor)).batch(batch_size=BATCH_SIZE)


In [115]:
# 훈련시키기
EPOCHS = 1

for epoch in range(EPOCHS):
    total_loss = 0
    
    dataset_count = tf.data.experimental.cardinality(train_dataset).numpy()
    tqdm_bar = tqdm(total=dataset_count)
    for step, (enc_batch, dec_batch) in enumerate(train_dataset):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_batch,
                    dec_batch,
                    transformer,
                    optimizer)

        total_loss += batch_loss

        tqdm_bar.set_description_str('Epoch %2d' % (epoch + 1))
        tqdm_bar.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (step + 1)))
        tqdm_bar.update()

  0%|          | 0/501 [00:00<?, ?it/s]

Tensor("Cast_3:0", shape=(64, 41), dtype=float32)
Tensor("Cast_3:0", shape=(64, 41), dtype=float32)
Tensor("Cast_3:0", shape=(46, 41), dtype=float32)


KeyboardInterrupt: ignored

In [ ]:
# # 번역 생성 함수

# def evaluate(sentence, model, src_tokenizer, tgt_tokenizer):
#     sentence = preprocess_sentence(sentence)

#     pieces = src_tokenizer.encode_as_pieces(sentence)
#     tokens = src_tokenizer.encode_as_ids(sentence)

#     _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
#                                                            maxlen=enc_train.shape[-1],
#                                                            padding='post')
    
#     ids = []
#     output = tf.expand_dims([tgt_tokenizer.bos_id()], 0)
#     for i in range(dec_train.shape[-1]):
#         enc_padding_mask, combined_mask, dec_padding_mask = \
#         generate_masks(_input, output)

#         predictions, enc_attns, dec_attns, dec_enc_attns =\
#         model(_input, 
#               output,
#               enc_padding_mask,
#               combined_mask,
#               dec_padding_mask)

#         predicted_id = \
#         tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

#         if tgt_tokenizer.eos_id() == predicted_id:
#             result = tgt_tokenizer.decode_ids(ids)
#             return pieces, result, enc_attns, dec_attns, dec_enc_attns

#         ids.append(predicted_id)
#         output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

#     result = tgt_tokenizer.decode_ids(ids)

#     return pieces, result, enc_attns, dec_attns, dec_enc_attns

In [ ]:
# 번역 생성 함수

def evaluate(sentence, model):
    sentence = preprocess_sentence(sentence)
    mecab = Mecab()
    
    
    pieces = mecab.morphs(sentence)
    tokens = get_encoded_sentence(sentence, word_to_index)

    # pieces =   src_tokenizer.encode_as_pieces(sentence)
    # tokens = src_tokenizer.encode_as_ids(sentence)

    _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')
    
    ids = []
    output = tf.expand_dims([word_to_index["<start>"]], 0)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input, 
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if word_to_index["<end>"] == predicted_id:
            result = get_decoded_sentence(ids, index_to_word)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = get_decoded_sentence(ids, index_to_word)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

In [ ]:
def translate(sentence, model):
    pieces, result, _, _, _ = \
    evaluate(sentence, model)
    
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    return result

In [ ]:
# # 학습

# from tqdm import tqdm_notebook 

# BATCH_SIZE = 64
# EPOCHS = 2

# examples = [
#     "지루하다, 놀러가고 싶어.",
#     "오늘 일찍 일어났더니 피곤하다.",
#     "간만에 여자친구랑 데이트 하기로 했어.",
#     "집에 있는다는 소리야."
# ]

# for epoch in range(EPOCHS):
#     total_loss = 0
    
#     idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
#     random.shuffle(idx_list)
#     t = tqdm_notebook(idx_list)

#     for (batch, idx) in enumerate(t):
#         batch_loss, enc_attns, dec_attns, dec_enc_attns = \
#         train_step(enc_train[idx:idx+BATCH_SIZE],
#                     dec_train[idx:idx+BATCH_SIZE],
#                     transformer,
#                     optimizer)

#         total_loss += batch_loss
        
#         t.set_description_str('Epoch %2d' % (epoch + 1))
#         t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

#     for example in examples:
#         translate(example, transformer)



In [108]:
examples = [
    "지루하다, 놀러가고 싶어.",
    "오늘 일찍 일어났더니 피곤하다.",
    "간만에 여자친구랑 데이트 하기로 했어.",
    "집에 있는다는 소리야."
]

In [109]:
for example in examples:
    translate(example, transformer)

Input: 지루하다, 놀러가고 싶어.
Predicted translation: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Input: 집에 있는다는 소리야.
Predicted translation: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <p

In [ ]:
translate('12시 땡!', transformer)

## STEP 7. 성능 측정하기

`calculate_bleu()` : 주어진 질문에 적절한 답변을 하는지 확인하고, BLEU Score를 계산 (챗봇 평가지표)




In [ ]:
def eval_bleu(src_corpus, tgt_corpus, verbose=True):
    total_score = 0.0
    sample_size = len(tgt_corpus)

    for idx in tqdm_notebook(range(sample_size)):
        src_tokens = src_corpus[idx]
        tgt_tokens = tgt_corpus[idx]
        
        src = []
        tgt = []
        
        for word in src_tokens:
            if word !=0 and word !=1 and word !=3 and word !=4:
                src.append(word)
        
        for word in tgt_tokens:
            if word != 0 and word != 3 and word !=4:
                tgt.append(word)

        src_sentence = get_decoded_sentence(src, index_to_word)
        tgt_sentence = get_decoded_sentence(tgt, index_to_word)
        
        
        reference = preprocess_sentence(tgt_sentence)
        candidate = translate(src_sentence, transformer)

        score = sentence_bleu([reference], candidate,
                              smoothing_function=SmoothingFunction().method1)
        total_score += score

        if verbose:
            print("Source Sentence: ", src_sentence)
            print("Model Prediction: ", candidate)
            print("Real: ", reference)
            print("Score: %lf\n" % score)

    print("Num of Sample:", sample_size)
    print("Total Score:", total_score / sample_size)

In [ ]:
eval_bleu(enc_val[::5], dec_val[::5], verbose=True)

# 회고

### - 이번 프로젝트에서 **어려웠던 점**.
> 해결 못한 부분
*  Loss 값이 자꾸 Nan으로 반환되어서 결과물도 \<pad>만 반복되었다.
도저히 원인을 찾을 수 없어서 마음이 아프다. 

### - 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**.
> 알아낸 점
* gensim 모델 불러와서 word2vec 실행해보기
* 한줄 코드 읽는법을 다시 한번 공부했다

> 아직 모호한점
*  bleu score 함수를 해당 task에 맞게  변형하는 법이 아직 이해가 조금  부족하다.



### - 루브릭 평가 지표를 맞추기 위해 **시도한 것들**.


>#### **루브릭평가 지표**
>|번호|평가문항|상세기준|
>|:---:|---|---|
>|1| 챗봇 훈련데이터 `전처리 과정`이 체계적으로 진행되었는가? |챗봇 훈련데이터를 위한 `전처리와 augmentation`이 적절히 수행되어 3만개 가량의 훈련데이터셋이 구축되었다.|
>|2| transformer 모델을 활용한 챗봇 모델이 `과적합을 피해 안정적`으로 훈련되었는가?| 과적합을 피할 수 있는 `하이퍼파라미터 셋`이 적절히 제시되었다.|
>|3|챗봇이 사용자의 `질문에 그럴듯한 형태로 답하는 사례`가 있는가?|주어진 예문을 포함하여 챗봇에 던진 질문에 적절히 답하는 사례가 제출되었다.|


### - **자기 다짐**

* 코드 복기

### - **참고**
>참고 깃허브
* https://github.com/kimddogyun/NLP_AIFFEL/blob/main/GD_12.ipynb
* https://github.com/jx-dohwan/Aiffel_NLP_Project/blob/main/%5BNLP_12%5DCreate_a_cool_chatbot.ipynb
* https://github.com/nevermet/AIFFEL/blob/master/G12_NiceChatBot.ipynb


